In [3]:
import optuna
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, make_scorer

In [4]:
train=pd.read_csv('/kaggle/input/fsajhd/df_train_clean.csv')
test=pd.read_csv('/kaggle/input/fsajhd/df_test_clean.csv')
sub=pd.read_csv('/kaggle/input/fsajhd/sample_submission.csv')

In [5]:
tarain=pd.read_csv('/kaggle/input/fsajhd/train.csv')

In [6]:
tarain.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [7]:
train.head(5)

,brand,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,price,Horsepower,Displacement,Cylinder Count,model_age
0,31,213000,2,0,312,71,2,1,4200,172.0,1.6,4.0,18
1,28,143250,2,0,263,10,0,1,4999,252.0,3.9,8.0,23
2,9,136731,1,0,38,71,2,1,13900,320.0,5.3,8.0,23
3,16,19500,2,2,29,14,2,1,45000,420.0,5.0,8.0,8
4,36,7388,2,0,29,10,2,1,97500,208.0,2.0,4.0,4


In [8]:
test.sample(5)

,brand,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,Horsepower,Displacement,Cylinder Count,model_age
119098,8,92000,2,0,29,14,2,1,420.000000,6.2,8.000000,4
91538,43,49000,2,2,304,14,2,1,440.000000,4.8,8.000000,10
121098,24,98995,2,0,93,71,2,1,340.010034,3.8,6.374268,6
98850,20,95428,2,0,128,71,0,1,265.000000,3.5,6.000000,10
109744,27,120000,2,0,38,10,0,1,272.000000,3.5,6.000000,17


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   brand           188533 non-null  int64  
 1   milage          188533 non-null  int64  
 2   fuel_type       188533 non-null  int64  
 3   transmission    188533 non-null  int64  
 4   ext_col         188533 non-null  int64  
 5   int_col         188533 non-null  int64  
 6   accident        188533 non-null  int64  
 7   clean_title     188533 non-null  int64  
 8   price           188533 non-null  int64  
 9   Horsepower      188533 non-null  float64
 10  Displacement    188533 non-null  float64
 11  Cylinder Count  188533 non-null  float64
 12  model_age       188533 non-null  int64  
dtypes: float64(3), int64(10)
memory usage: 18.7 MB


## XgBoost

In [10]:
# import optuna
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split, cross_val_score
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.impute import SimpleImputer
# from xgboost import XGBRegressor
# from sklearn.metrics import mean_squared_error, make_scorer

# # Load dataset (Ensure 'train' is already loaded)
# X = train.drop(columns=['price'])
# y = train['price']

# # Split dataset
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# # Preprocessing (Standard Scaling for numerical features)
# numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', StandardScaler(), numeric_features)
#     ])

# # RMSE scorer for optimization
# rmse_scorer = make_scorer(mean_squared_error, squared=False)

# # Define the objective function for Optuna
# def objective(trial):
#     params = {
#         'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=50),
#         'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
#         'max_depth': trial.suggest_int('max_depth', 3, 12),
#         'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
#         'subsample': trial.suggest_float('subsample', 0.5, 1.0),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
#         'gamma': trial.suggest_float('gamma', 0, 0.5),
#         'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
#         'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
#         'tree_method': 'gpu_hist',  # Enable GPU
#         'predictor': 'gpu_predictor',
#         'random_state': 42
#     }

#     # Create pipeline
#     pipeline = Pipeline(steps=[
#         ('preprocessor', preprocessor),
#         ('model', XGBRegressor(**params))
#     ])

#     # Perform cross-validation
#     scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring=rmse_scorer)
    
#     return np.mean(scores)

# # Run Optuna optimization
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=50)

# # Get best parameters
# best_params = study.best_params
# print("Best Hyperparameters:", best_params)

# -------------------------------------Output---------------------------------------------------

# Best Hyperparameters: {'n_estimators': 200,
#                        'learning_rate': 0.026075961680376267, 
#                        'max_depth': 7, 'min_child_weight': 8,
#                        'subsample': 0.6403035387419401, 
#                        'colsample_bytree': 0.5532517223827789, 
#                        'gamma': 0.143179155034863,
#                        'reg_alpha': 2.1450990765269337e-07, 
#                        'reg_lambda': 0.05942705889240155}

In [11]:
# import joblib
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import KFold
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import StandardScaler
# from xgboost import XGBRegressor
# from sklearn.metrics import mean_squared_error

# # Load dataset
# X = train.drop(columns=['price'])
# y = train['price']

# # Define numerical and categorical columns
# num_cols = ['milage', 'Horsepower', 'Displacement', 'Cylinder Count', 'model_age']
# cat_cols = ['brand', 'fuel_type', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title']

# # Updated hyperparameters from Optuna with GPU support
# tuned_params = {
#     'n_estimators': 200,
#     'learning_rate': 0.026075961680376267,
#     'max_depth': 7,
#     'min_child_weight': 8,
#     'subsample': 0.6403035387419401,
#     'colsample_bytree': 0.5532517223827789,
#     'gamma': 0.143179155034863,
#     'reg_alpha': 2.1450990765269337e-07,
#     'reg_lambda': 0.05942705889240155,
#     'tree_method': 'gpu_hist',  # Enable GPU support
#     'random_state': 42
# }

# # Define preprocessing pipeline
# num_pipeline = Pipeline(steps=[('scaler', StandardScaler())])
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num_pipeline', num_pipeline, num_cols)
#     ],
#     remainder='passthrough'
# )

# # Number of folds for cross-validation
# n_folds = 5
# kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

# oof_preds = np.zeros(len(X))
# oof_df = pd.DataFrame(columns=['ID', 'Actual', 'OOF_Pred_XGBoost', 'Fold'])

# for fold, (train_idx, val_idx) in enumerate(kf.split(X, y), start=1):
#     print(f"\nTraining Fold {fold}/{n_folds}...")
    
#     X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
#     y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
#     # Create pipeline with preprocessor and XGBoost model
#     pipeline = Pipeline(steps=[
#         ('preprocessor', preprocessor),
#         ('model', XGBRegressor(**tuned_params))
#     ])
    
#     # Train model
#     pipeline.fit(X_tr, y_tr)
    
#     # Predict on validation set
#     y_val_pred = pipeline.predict(X_val)
#     oof_preds[val_idx] = y_val_pred
    
#     # Compute fold RMSE
#     fold_rmse = mean_squared_error(y_val, y_val_pred, squared=False)
#     print(f"Fold {fold} RMSE: {fold_rmse:.4f}")
    
#     # Store fold results in DataFrame
#     fold_df = pd.DataFrame({
#         'ID': X.index[val_idx],
#         'Actual': y_val.values,
#         'OOF_Pred_XGBoost': y_val_pred,
#         'Fold': fold
#     })
    
#     oof_df = pd.concat([oof_df, fold_df], ignore_index=True)

# # Compute overall RMSE
# oof_rmse = mean_squared_error(y, oof_preds, squared=False)
# print(f"\nOverall OOF RMSE: {oof_rmse:.4f}")

# # Save OOF predictions
# oof_df.to_csv('oof_predictions_xgboost.csv', index=False)
# print("OOF predictions saved to 'oof_predictions_xgboost.csv'.")

# # Save trained model
# joblib.dump(pipeline, 'xgboost_model.pkl')
# print("Model saved as 'xgboost_model.pkl'.")

# # -------------------------------------Output---------------------------------------------------

# Overall OOF RMSE: 72832.0142

In [12]:
# final_test_predictions = pipeline.predict(test)

# # Save submission file
# sub = pd.read_csv('/kaggle/input/fsajhd/sample_submission.csv')
# sub['price'] = final_test_predictions
# sub.to_csv('XgBoost_submission.csv', index=False)
# print("Submission file saved as 'lrsubmission.csv'.")
# sub.head()


## Cat Boost

In [13]:
# import optuna
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split, cross_val_score, KFold
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from catboost import CatBoostRegressor
# from sklearn.metrics import mean_squared_error, make_scorer

# # Load dataset
# X = train.drop(columns=['price'])
# y = train['price']

# # Split dataset
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# # Define numerical and categorical columns
# num_cols = ['milage', 'Horsepower', 'Displacement', 'Cylinder Count', 'model_age']
# cat_cols = ['brand', 'fuel_type', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title']

# # Preprocessing (Standard Scaling for numerical features)
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', StandardScaler(), num_cols)
#     ],
#     remainder='passthrough'
# )

# # RMSE scorer for optimization
# rmse_scorer = make_scorer(mean_squared_error, squared=False)

# # Define objective function for Optuna
# def objective(trial):
#     params = {
#         'iterations': trial.suggest_int('iterations', 100, 1000, step=100),
#         'depth': trial.suggest_int('depth', 3, 10),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
#         'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 1e-1, log=True),
#         'random_seed': 42,
#         'loss_function': 'RMSE',
#         'verbose': 0
#     }

#     # Create pipeline
#     pipeline = Pipeline(steps=[
#         ('preprocessor', preprocessor),
#         ('model', CatBoostRegressor(**params))
#     ])

#     # Perform cross-validation
#     kf = KFold(n_splits=5, shuffle=True, random_state=42)
#     scores = cross_val_score(pipeline, X_train, y_train, cv=kf, scoring=rmse_scorer)
    
#     return np.mean(scores)

# # Run Optuna optimization
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=50)

# # Get best parameters
# best_params = study.best_params
# print("Best Hyperparameters:", best_params)

# -------------------------------------Output---------------------------------------------------

# Best Hyperparameters: {'iterations': 500,
#                        'depth': 9,
#                        'learning_rate': 0.012904818994335723,
#                        'l2_leaf_reg': 0.009754632381874578}

In [14]:
train.head(
)

,brand,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,price,Horsepower,Displacement,Cylinder Count,model_age
0,31,213000,2,0,312,71,2,1,4200,172.0,1.6,4.0,18
1,28,143250,2,0,263,10,0,1,4999,252.0,3.9,8.0,23
2,9,136731,1,0,38,71,2,1,13900,320.0,5.3,8.0,23
3,16,19500,2,2,29,14,2,1,45000,420.0,5.0,8.0,8
4,36,7388,2,0,29,10,2,1,97500,208.0,2.0,4.0,4


In [15]:
# import joblib
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import KFold
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import StandardScaler
# from catboost import CatBoostRegressor
# from sklearn.metrics import mean_squared_error

# # Load dataset
# X = train.drop(columns=['price'])
# y = train['price']

# # Define numerical and categorical columns
# num_cols = ['milage', 'Horsepower', 'Displacement', 'Cylinder Count', 'model_age']
# cat_cols = ['brand', 'fuel_type', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title']

# # Updated hyperparameters from Optuna with GPU support
# tuned_params = {
#     'iterations': 500,
#     'depth': 9,
#     'learning_rate': 0.012904818994335723,
#     'l2_leaf_reg': 0.009754632381874578,
#     'random_seed': 42,
#     'loss_function': 'RMSE',
#     'verbose': 100,
#     'task_type': 'GPU'
# }

# # Define preprocessing pipeline
# num_pipeline = Pipeline(steps=[('scaler', StandardScaler())])
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num_pipeline', num_pipeline, num_cols)
#     ],
#     remainder='passthrough'
# )

# # Number of folds for cross-validation
# n_folds = 5
# kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

# oof_preds = np.zeros(len(X))

# oof_df = pd.DataFrame(columns=['ID', 'Actual', 'OOF_Pred_CatBoost', 'Fold'])

# for fold, (train_idx, val_idx) in enumerate(kf.split(X, y), start=1):
#     print(f"\nTraining Fold {fold}/{n_folds}...")
    
#     X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
#     y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
#     # Create pipeline with preprocessor and CatBoost model
#     pipeline = Pipeline(steps=[
#         ('preprocessor', preprocessor),
#         ('model', CatBoostRegressor(**tuned_params))
#     ])
    
#     # Train model
#     pipeline.fit(X_tr, y_tr)
    
#     # Predict on validation set
#     y_val_pred = pipeline.predict(X_val)
#     oof_preds[val_idx] = y_val_pred
    
#     # Compute fold RMSE
#     fold_rmse = mean_squared_error(y_val, y_val_pred, squared=False)
#     print(f"Fold {fold} RMSE: {fold_rmse:.4f}")
    
#     # Store fold results in DataFrame
#     fold_df = pd.DataFrame({
#         'ID': X.index[val_idx],
#         'Actual': y_val.values,
#         'OOF_Pred_CatBoost': y_val_pred,
#         'Fold': fold
#     })
    
#     oof_df = pd.concat([oof_df, fold_df], ignore_index=True)

# # Compute overall RMSE
# oof_rmse = mean_squared_error(y, oof_preds, squared=False)
# print(f"\nOverall OOF RMSE: {oof_rmse:.4f}")

# # Save OOF predictions
# oof_df.to_csv('oof_predictions_catboost.csv', index=False)
# print("OOF predictions saved to 'oof_predictions_catboost.csv'.")

# # Save trained model
# joblib.dump(pipeline, 'catboost_model.pkl')
# print("Model saved as 'catboost_model.pkl'.")

# -------------------------------------Output---------------------------------------------------

#Overall OOF RMSE: 73119.5593

In [16]:
# final_test_predictions = pipeline.predict(test)

# # Save submission file
# sub = pd.read_csv('/kaggle/input/fsajhd/sample_submission.csv')
# sub['price'] = final_test_predictions
# sub.to_csv('CatBoost_submission.csv', index=False)
# print("Submission file saved as 'lrsubmission.csv'.")
# sub.head()


## Aada Boost

In [17]:
# import optuna
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split, cross_val_score
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.impute import SimpleImputer
# from sklearn.ensemble import AdaBoostRegressor
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.metrics import mean_squared_error, make_scorer

# # Load dataset
# X = train.drop(columns=['price'])
# y = train['price']

# # Split dataset
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# # Preprocessing (Standard Scaling for numerical features)
# numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', StandardScaler(), numeric_features)
#     ])

# # RMSE scorer for optimization
# rmse_scorer = make_scorer(mean_squared_error, squared=False)

# # Define the objective function for Optuna
# def objective(trial):
#     params = {
#         'n_estimators': trial.suggest_int('n_estimators', 50, 500, step=50),
#         'learning_rate': trial.suggest_float('learning_rate', 0.001, 1.0, log=True),
#         'max_depth': trial.suggest_int('max_depth', 1, 10),
#         'random_state': 42
#     }
    
#     base_estimator = DecisionTreeRegressor(max_depth=params['max_depth'])
#     model = AdaBoostRegressor(
#         estimator=base_estimator,
#         n_estimators=params['n_estimators'],
#         learning_rate=params['learning_rate'],
#         random_state=params['random_state']
#     )
    
#     pipeline = Pipeline(steps=[
#         ('preprocessor', preprocessor),
#         ('model', model)
#     ])
    
#     scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring=rmse_scorer)
    
#     return np.mean(scores)

# # Run Optuna optimization
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=50)

# # Get best parameters
# best_params = study.best_params
# print("Best Hyperparameters:", best_params)

# -------------------------------------Output---------------------------------------------------

# best_params = {
#     'n_estimators': 100,
#     'learning_rate': 0.0030416565304804866,
#     'max_depth': 9,
#     'random_state': 42
# }


In [18]:
import joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# Load dataset
X = train.drop(columns=['price'])
y = train['price']

# Define best hyperparameters from Optuna
best_params = {
    'n_estimators': 100,
    'learning_rate': 0.0030416565304804866,
    'max_depth': 9,
    'random_state': 42
}

# Define preprocessing pipeline
num_pipeline = Pipeline(steps=[('scaler', StandardScaler())])
preprocessor = ColumnTransformer(
    transformers=[
        ('num_pipeline', num_pipeline, X.select_dtypes(include=['int64', 'float64']).columns)
    ],
    remainder='passthrough'
)

# Number of folds for cross-validation
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

oof_preds = np.zeros(len(X))
oof_df = pd.DataFrame(columns=['ID', 'Actual', 'OOF_Pred_AdaBoost', 'Fold'])

for fold, (train_idx, val_idx) in enumerate(kf.split(X, y), start=1):
    print(f"\nTraining Fold {fold}/{n_folds}...")
    
    X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    # Create pipeline with preprocessor and AdaBoost model
    base_estimator = DecisionTreeRegressor(max_depth=best_params['max_depth'])
    model = AdaBoostRegressor(
        base_estimator=base_estimator,
        n_estimators=best_params['n_estimators'],
        learning_rate=best_params['learning_rate'],
        random_state=best_params['random_state'])
    
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    
    # Train model
    pipeline.fit(X_tr, y_tr)
    
    # Predict on validation set
    y_val_pred = pipeline.predict(X_val)
    oof_preds[val_idx] = y_val_pred
    
    # Compute fold RMSE
    fold_rmse = mean_squared_error(y_val, y_val_pred, squared=False)
    print(f"Fold {fold} RMSE: {fold_rmse:.4f}")
    
    # Store fold results in DataFrame
    fold_df = pd.DataFrame({
        'ID': X.index[val_idx],
        'Actual': y_val.values,
        'OOF_Pred_AdaBoost': y_val_pred,
        'Fold': fold
    })
    
    oof_df = pd.concat([oof_df, fold_df], ignore_index=True)

# Compute overall RMSE
oof_rmse = mean_squared_error(y, oof_preds, squared=False)
print(f"\nOverall OOF RMSE: {oof_rmse:.4f}")

# Save OOF predictions
oof_df.to_csv('oof_predictions_adaboost.csv', index=False)
print("OOF predictions saved to 'oof_predictions_adaboost.csv'.")

# Save trained model
joblib.dump(pipeline, 'adaboost_model.pkl')
print("Model saved as 'adaboost_model.pkl'.")


Training Fold 1/5...


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Fold 1 RMSE: 70216.4987

Training Fold 2/5...


<ipython-input-18-073f9eced56a>:78: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  oof_df = pd.concat([oof_df, fold_df], ignore_index=True)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Fold 2 RMSE: 69142.7205

Training Fold 3/5...


...

Fold 3 RMSE: 76075.6094

Training Fold 4/5...


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Fold 4 RMSE: 77008.5131

Training Fold 5/5...


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Fold 5 RMSE: 76917.5347

Overall OOF RMSE: 73952.9107
OOF predictions saved to 'oof_predictions_adaboost.csv'.
Model saved as 'adaboost_model.pkl'.


In [19]:
final_test_predictions = pipeline.predict(test)

# Save submission file
sub = pd.read_csv('/kaggle/input/fsajhd/sample_submission.csv')
sub['price'] = final_test_predictions
sub.to_csv('aadaBoost_submission.csv', index=False)
print("Submission file saved as 'aadasubmission.csv'.")
sub.head()


Submission file saved as 'aadasubmission.csv'.


,id,price
0,188533,16304.292352
1,188534,83574.507495
2,188535,49196.920144
3,188536,25094.532374
4,188537,30644.890365
